In [1]:
import numpy as np
from utilities.Dataset import *
import pickle
import sys
from typing import List
import torch
import os

In [2]:
participant_data_path = './EEG/number'
processed_data_path = './EEG/processed_data'
split_data_path = './EEG/split_data'

In [3]:
ps = os.listdir(participant_data_path)
participants=[]
for p in ps:
    p= p[:3]
    if p not in participants:
        participants.append(p)
        
participants= sorted(participants)
# participants, len(participants)

In [4]:
load_name=f"userfold_data_scaled_p_dictionary-number"
data_dir = "./EEG/split_data/standard_scaled"
try:
    raw_user_fold= pickle.load(open(os.path.join(data_dir, f"{load_name}.pkl"), "rb"))
except:
    print(f"pickle file does not exist. Use EEG-Preprocess.ipynb and EEG-Split.ipynb to save data setting.")
    sys.exit()

In [5]:
from utilities.userfold_framework import *
from Models.AR_EEG_models import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List



In [ ]:
from torcheeg.models import EEGNet
import Models.model_func as Model_Func
from torch import nn
from Models.multi_models import *

DEVICE= torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

learning_rate = 0.00005
batch_size = 64
n_epochs = 300
transpose_channels=True
participants_dictionary=[]
# participants_online_dictionary=[]
participants_grads_dictionary={}
b_acc_list=[]
c0_acc_list=[]
c1_acc_list=[]

# EPOCH=[
    
# ]

for i in range(len(participants)):

    train_dataloader, val_dataloader, classes, input_dim, class_ratio= user_fold_load(i,
                                                                                      raw_user_fold,
                                                                                      participants,
                                                                                      batch_size=batch_size,
                                                                                      transpose_channels=transpose_channels)

    eegnet= EEGNet(
        chunk_size=input_dim[1],
        num_electrodes=input_dim[0],
        num_classes=classes,
        kernel_1= 32,
        kernel_2=32,
        F1=8,
        F2=16,
        dropout=0.5
    ).to(DEVICE)
    
    classifier= LSTM_EEGNet_Wrapper(DEVICE, eegnet, input_dim).to(DEVICE)
#     optimizer= torch.optim.RMSprop(classifier.parameters(), lr=learning_rate)
    optimizer = torch.optim.Adam(classifier.parameters(), lr = learning_rate)

    
    criterion= torch.nn.CrossEntropyLoss(weight=torch.tensor(class_ratio, dtype=torch.float).to(DEVICE))
#     criterion = nn.NLLLoss(weight=torch.tensor(class_ratio, dtype=torch.float).to(DEVICE))
    
    scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)
    
    saved_dir= "./EEG/saved_models/Userfold/run1"
    model= EEGNet_IE_HP_Wrapper(
        DEVICE, classifier, input_dim
    ).to(DEVICE)

    train_func= eeg_grad_train
    model.training_procedure(iteration=n_epochs,
                             train_dataloader=train_dataloader,
                             val_dataloader=val_dataloader,
                             print_cycle=2,
                             path=f"./dictionary/intermdiate_dicts",
                             loss_func=criterion,
                             optimiser=optimizer, #scheduler=scheduler,
                             train_func=train_func
                            )
    if model.epoch == n_epochs+1:
        EPOCH= n_epochs
    else:
        EPOCH= model.epoch
    
    torch.save(model.state_dict(), 
           os.path.join(
               saved_dir, f"Userfold-{participants[i]}-LSTM_EEGNet-IE_Multivariate-e{EPOCH}.pt"
           )
    )
    pickle.dump(model.return_IE_grad(),
                open(
                    os.path.join(
                        saved_dir, f"Userfold-{participants[i]}-LSTM_EEGNet-IE_Multivariate-grad_all-e{EPOCH}.pkl"
                    ), "wb")
    )
# OR
#     model.load_state_dict(
#     torch.load(
#         open(
#             os.path.join(
#                 saved_dir, f"Userfold-{participants[i]}-EEGNet-IE_Multivariate-e{n_epochs}.pt"
#             ), "rb"
#         )
#               )
#     )
#     model.IE_grad= pickle.load(
#             open(
#                 os.path.join(
#                     saved_dir, f"Userfold-{participants[i]}-EEGNet-IE_Multivariate-grad_all-e{n_epochs}.pkl"
#                 ),"rb"
#             )
#         )
    
    
    prediction, dictionary= model.prediction_procedure(val_dataloader, dict_flag=True)
    
    ys= np.concatenate([y.detach().cpu().numpy() for x, y in val_dataloader])
    
    c0_acc, c1_acc, b_acc= calculate_accuracy(ys, prediction)
    print("c0_acc", c0_acc, ", c1_acc", c1_acc, ", b_acc", b_acc)
    b_acc_list.append(b_acc)
    c0_acc_list.append(c0_acc)
    c1_acc_list.append(c1_acc)
#     participants_grads_dictionary[participants[i]]= model.return_IE_grad()
    participants_dictionary.append(dictionary)
    
   

tmp=[]
for i, dictionary in enumerate(participants_dictionary):
    print(f"User {participants[i]} f1: {dictionary['weighted avg']['f1-score']} acc: {dictionary['accuracy']}")
    print(f" c0: {c0_acc_list[i]} c1: {c1_acc_list[i]} bacc: {b_acc_list[i]}")
    tmp.append(dictionary['weighted avg']['f1-score'])

print(f"average {np.mean(tmp)}")
print()
print(np.array(b_acc_list).mean())
print(np.array(c1_acc_list).mean())
print(np.array(c0_acc_list).mean())


Iterations:   0%|                                           | 0/300 [00:00<?, ?it/s]

Epoch  0 , loss 0.6200654319688386


Iterations:   0%|                                   | 1/300 [00:03<17:42,  3.55s/it]

Epoch:  0
t_loss:  0.6200654319688386 , v_loss:  0.6855846444765726
t_acc:  0.61531279178338 , v_acc:  0.6894409937888198
t_recall:  0.4862745501708733 , v_recall:  0.5
t_prec:  0.4799191585744922 , v_prec:  0.3447204968944099
t_f:  0.47268616334764196 , v_f:  0.4080882352941176
////////


Iterations:   1%|▏                                  | 2/300 [00:04<10:19,  2.08s/it]

...

Epoch  281 , loss 0.36813730537891387
Epoch  282 , loss 0.37168634951114654


Iterations:  94%|███████████████████████████████▏ | 283/300 [07:48<00:28,  1.67s/it]

Epoch:  282
t_loss:  0.37168634951114654 , v_loss:  0.6748305410146713
t_acc:  0.7596485723250705 , v_acc:  0.7112299465240641
t_recall:  0.621266570056387 , v_recall:  0.5689749182115594
t_prec:  0.7910641025641025 , v_prec:  0.6393939393939394
t_f:  0.6264678160708492 , v_f:  0.5626299376299375
////////


Iterations:  95%|███████████████████████████████▏ | 284/300 [07:50<00:25,  1.60s/it]

Epoch  283 , loss 0.3772492092847824
Epoch  284 , loss 0.37127041786909104


In [ ]:
userfold_results_summary(participants_dictionary, participants)
userfold_classwise_results_summary(participants_dictionary, participants)


In [ ]:
participant_grad={"AUC":[],"ROC":[],"STD":[]}
for i in range(len(participants)):
    g= pickle.load(
            open(
                os.path.join(
                    saved_dir, f"Userfold-{participants[i]}-LSTM_EEGNet-IE_Multivariate-grad_all-e{n_epochs}.pkl"
                ),"rb"
            )
    )
    
    participant_grad["AUC"].append(
        pd.DataFrame(
            Grad_AUC_with_multivar_grad(
                g, n_epochs, np.zeros(input_dim), x_shape=0, scaling=True
            )
        ).to_numpy()
    )
    participant_grad["ROC"].append(
          pd.DataFrame(
            Grad_ROC_with_multivar_grad(
                g, n_epochs, np.zeros(input_dim), x_shape=0, scaling=True
                                       )
          ).to_numpy()
    )
    participant_grad["STD"].append(
        pd.DataFrame(
            Grad_STD_with_multivar_grad(
                g, n_epochs, np.zeros(input_dim), x_shape=0, scaling=True
                                       )
        ).to_numpy()
    )
    
avg_grad={
    "AUC":np.array(participant_grad["AUC"]).mean(0),
    "ROC":np.array(participant_grad["ROC"]).mean(0),
    "STD":np.array(participant_grad["STD"]).mean(0)
}

In [ ]:
start=-0.5
step=1.5/(188-1)
timestep_labels=[]
for i in range(input_dim[1]):
    timestep_labels.append(round(start+step*i,3))
    
# timestep_labels

In [ ]:
# channel_names=[i for i in range(input_dim[0])]
channel_names=["AFz","F3","F1","Fz","F2","F4","FC5","FC3","FC1","FCz","FC2",
               "FC4","FC6","C5","C3","C1","Cz","C2","C4","C6","CP5","CP3",
               "CP1","CPz","CP2","CP4","CP6","P3","P1","Pz","P2","P4"]

In [ ]:
# from utilities.subset_func import *
# from utilities.utility_func import return_channels_index, return_timestep_labels

# channels_index= return_channels_index()
# timestep_labels= return_timestep_labels()
fig, ax= plt.subplots(3,1)

# x_shape=0
# dictionaries={"AUC":[],"ROC":[],"STD":[]}
# for idx,p in enumerate(participants):
#     dictionary= Grad_AUC_with_multivar_grad(participants_grads_dictionary[p],
#                                             n_epochs,
#                                             X=np.zeros(input_dim),
#                                             x_shape=x_shape
#                                            )
#     dictionaries["AUC"].append(pd.DataFrame(dictionary).to_numpy())
# #     sns.heatmap(pd.DataFrame(dictionary), 
# #                 xticklabels=timestep_labels,
# #                 yticklabels=channels_index,
# #                 annot=False, ax=ax[idx], cbar_kws={"pad":0.02})
#     dictionary= Grad_ROC_with_multivar_grad(participants_grads_dictionary[p],
#                                             n_epochs,
#                                             np.zeros(input_dim),
#                                             x_shape=x_shape
#                                            )
#     dictionaries["ROC"].append(pd.DataFrame(dictionary).to_numpy())

#     dictionary= Grad_STD_with_multivar_grad(participants_grads_dictionary[p],
#                                             n_epochs,
#                                             np.zeros(input_dim),
#                                             x_shape=x_shape
#                                            )
#     dictionaries["STD"].append(pd.DataFrame(dictionary).to_numpy())


#     ax[idx].set_ylabel(f"{p}")

sns.heatmap(pd.DataFrame(avg_grad["AUC"]), 
            xticklabels=timestep_labels,
            yticklabels=channel_names, ax=ax[0],
            annot=False, cbar_kws={"pad":0.02})

ax[0].set_title("AUC")

sns.heatmap(pd.DataFrame(avg_grad["ROC"]), 
            xticklabels=timestep_labels,
            yticklabels=channel_names, ax=ax[1],
            annot=False, cbar_kws={"pad":0.02})
ax[1].set_title("ROC")

sns.heatmap(pd.DataFrame(avg_grad["STD"]), 
            xticklabels=timestep_labels,
            yticklabels=channel_names, ax=ax[2],
            annot=False, cbar_kws={"pad":0.02})
ax[2].set_title("STD")

# plt.suptitle("Relative Importance Estimate of Time Series")
fig.set_figwidth(20)
fig.set_figheight(50)
plt.tight_layout()

In [ ]:
# timestep_labels[99:99+62]

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(40,10))

# for i in range(len(methods)):
#     scaler= MinMaxScaler()
#     df= pd.DataFrame(np.array(dictionaries[methods[i]]).mean(axis=0))

#     # #     channel_nonzero_count= np.count_nonzero(df.to_numpy()>threshold, axis=1)

#     # #     df_with_zeros= pd.DataFrame(channel_nonzero_count)
#     df.index= channel_names
#     df.columns=timestep_labels
    
#     df_list["time"][methods[i]]=df.sum().to_numpy().reshape(-1)
# #     nonzeros_list= (df_with_zeros>0).to_numpy().reshape(-1)
# #     df= df_with_zeros[nonzeros_list]
#     scaled= scaler.fit_transform(df.sum().to_numpy().reshape(-1,1))
#     plt.plot(scaled)
# plt.legend(methods)
# plt.xlabel("Timesteps")
# plt.axvspan(112,128, color="red", alpha=0.2)
# plt.axvspan(142,150, color="red", alpha=0.2)
# plt.axvspan(157,159, color="red", alpha=0.2)
# plt.xticks([i for i in range(len(timestep_labels))],timestep_labels, rotation="vertical")
# plt.ylabel("Normalised Importance")
# plt.margins(x=0)
# plt.tight_layout()
# #     sns.heatmap(df.sum().to_numpy().reshape(-1,1),annot=True, 
# #                 yticklabels=timestep_labels, ax=ax[i][1],
# #                 xticklabels=False, cbar_kws={"pad":0.02})

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, RobustScaler
plt.figure(figsize=(8,6))
methods=["AUC", "ROC", "STD"]
for i in methods:
    scaler= MinMaxScaler()
    scaled_avg_grad= scaler.fit_transform(avg_grad[i].sum(0).reshape(-1,1))
    df= pd.DataFrame(scaled_avg_grad)    
#     df.index= channel_names
#     df.columns=timestep_labels
    
#     scaled= scaler.fit_transform(df.sum().to_numpy().reshape(-1,1))
    plt.plot(scaled_avg_grad)
plt.legend(methods)
plt.xlabel("Timesteps")
plt.axvspan(107,121, color="green", alpha=0.2)
plt.axvspan(112,128, color="red", alpha=0.2)
plt.axvspan(142,150, color="red", alpha=0.2)
plt.axvspan(157,159, color="red", alpha=0.2)
plt.xticks([0,31,62,93,124,155,187],[-0.5, -0.25, 0, 0.25, 0.5, 0.75, 1.0])
plt.ylabel("Normalised Importance")
plt.margins(x=0)
matplotlib.rcParams.update({"font.size":18})
plt.tight_layout()


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, RobustScaler
plt.figure(figsize=(8,6))
methods=["AUC", "ROC", "STD"]
for i in methods:
    scaler= RobustScaler()
    scaled_avg_grad= scaler.fit_transform(avg_grad[i].sum(0).reshape(-1,1))
    df= pd.DataFrame(scaled_avg_grad)    
#     df.index= channel_names
#     df.columns=timestep_labels
    
#     scaled= scaler.fit_transform(df.sum().to_numpy().reshape(-1,1))
    plt.plot(scaled_avg_grad)
plt.legend(methods)
plt.xlabel("Timesteps")
plt.axvspan(107,121, color="green", alpha=0.2)
plt.axvspan(112,128, color="red", alpha=0.2)
plt.axvspan(142,150, color="red", alpha=0.2)
plt.axvspan(157,159, color="red", alpha=0.2)
plt.xticks([0,31,62,93,124,155,187],[-0.5, -0.25, 0, 0.25, 0.5, 0.75, 1.0])
plt.ylabel("Normalised Importance")
plt.margins(x=0)
matplotlib.rcParams.update({"font.size":18})
plt.tight_layout()


In [ ]:
# coordinates= np.array([
#     [0.0002313, 0.080771], #AFz
#     [-0.0502438,  0.0531112], #F3
#     [-0.0274958,  0.0569311], #F1
#     [0.0003122, 0.058512], #Fz
#     [0.0295142, 0.0576019], #F2
#     [0.0518362, 0.0543048], #F4
#     [-0.0772149,  0.0186433], #FC5
#     [-0.0601819,  0.0227162], #FC3
#     [-0.0340619,  0.0260111], #FC1
#     [0.0003761, 0.02739], #FCz
#     [0.0347841, 0.0264379], # FC2
#     [0.0622931, 0.0237228], #FC4
#     [0.0795341, 0.0199357], #FC6
#     [-0.0802801, -0.0137597], #C5
#     [-0.0653581, -0.0116317], # C3
#     [-0.036158 , -0.0099839], # C1
#     [0.0004009, -0.009167], # Cz
#     [0.037672 , -0.0096241], # C2
#     [0.0671179, -0.0109003], #C4
#     [0.0834559, -0.0127763], #C6
#     [-0.0795922, -0.0465507], #CP5
#     [-0.0635562, -0.0470088], #CP3
#     [-0.0355131, -0.0472919], #CP1
#     [0.0003858, -0.047318], #CPz
#     [0.0383838, -0.0470731], #CP2
#     [0.0666118, -0.0466372], #CP4
#     [0.0833218, -0.0461013], #CP6
#     [-0.0530073, -0.0787878], #P3
#     [-0.0286203, -0.0805249], #P1
#     [0.0003247, -0.081115], #Pz
#     [0.0319197, -0.0804871], #P2
#     [0.0556667, -0.0785602], #P4
# ])
# x=coordinates[:,0]
# y=coordinates[:,1]

In [ ]:
# for i in range(len(coordinates)):
#     plt.scatter(x[i],y[i])
# # plt.legend(channel_names)

In [ ]:
# _=mne.channels.make_standard_montage("standard_1020")
# _.get_positions()

In [ ]:
# import mne
# _=mne.channels.make_standard_montage("standard_1020").plot(sphere=0.11)

In [ ]:
# import matplotlib
# fig= plt.figure()
# ax= plt.axes((0,0,1.5,1.5))
# im, _= mne.viz.plot_topomap(
#     df.T.sum().to_numpy().reshape(-1),
#     coordinates,
#     ch_type= "eeg",
#     sensors=True,
#     names=channel_names,
#     cmap="Blues",
#     axes=ax,
#     show=False
# #     res=4000
# )
# fig.add_axes(ax)
# cbar_ax= fig.add_axes([1.3,0.2, 0.1,1])
# clb= fig.colorbar(im, cax=cbar_ax)

# for tt in plt.findobj(fig, matplotlib.text.Text):
#     if tt.get_text() in channel_names:
#         tt.set_fontsize(12)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import matplotlib
import mne
info= mne.create_info(channel_names, sfreq=500, ch_types=32*["eeg"])
info.set_montage("standard_1020")

fig, ax= plt.subplots(2,3)
# ax= plt.axes((0,0,1.5,1.5))
for idx, i in enumerate(methods):
    scaler= MinMaxScaler()
    scaled_avg_grad= scaler.fit_transform(avg_grad[i].sum(1).reshape(-1,1))
#     scaled_avg_grad= avg_grad[i].sum(1)

    minmax_im, _= mne.viz.plot_topomap(
        scaled_avg_grad.reshape(-1),
        info,
        ch_type= "eeg",
        sensors=True,
        names=channel_names,
        cmap="Blues",
        axes=ax[0][idx],
        show=False,
        extrapolate="local"
        #,vlim=(0,1)
    #     sphere=0.09
    )
    ax[0][idx].set_title(f"MinMax: {i}")
    
for idx, i in enumerate(methods):
    scaler= RobustScaler()
    scaled_avg_grad= scaler.fit_transform(avg_grad[i].sum(1).reshape(-1,1))
#     scaled_avg_grad= avg_grad[i].sum(1)

    robust_im, _= mne.viz.plot_topomap(
        scaled_avg_grad.reshape(-1),
        info,
        ch_type= "eeg",
        sensors=True,
        names=channel_names,
        cmap="Blues",
        axes=ax[1][idx],
        show=False,
        extrapolate="local",
#         vlim=(0,None)
    #     sphere=0.09
    )
    ax[1][idx].set_title(f"Robust: {i}")
    
fig.set_figheight(25)
fig.set_figwidth(25)
# fig.add_axes(ax)
cbar_ax= fig.add_axes([.92,0.66, 0.02,0.18])
clb= fig.colorbar(minmax_im, cax=cbar_ax)
clb.set_label("Importance Estimate", rotation=270,labelpad=20, fontsize=18)

cbar_ax= fig.add_axes([.92,0.22, 0.02,0.18])
clb= fig.colorbar(robust_im, cax=cbar_ax)
clb.set_label("Importance Estimate", rotation=270,labelpad=20, fontsize=18)

for tt in plt.findobj(fig, matplotlib.text.Text):
    if tt.get_text() in channel_names:
        tt.set_fontsize(14)
# plt.tight_layout()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import matplotlib
import mne
info= mne.create_info(channel_names, sfreq=500, ch_types=32*["eeg"])
info.set_montage("standard_1020")

fig, ax= plt.subplots(2,3)
# ax= plt.axes((0,0,1.5,1.5))
for idx, i in enumerate(methods):
    scaler= MinMaxScaler()
    scaled_avg_grad= scaler.fit_transform(avg_grad[i].sum(1).reshape(-1,1))
#     scaled_avg_grad= avg_grad[i].sum(1)

    minmax_im, _= mne.viz.plot_topomap(
        scaled_avg_grad.reshape(-1),
        info,
        ch_type= "eeg",
        sensors=True,
        names=channel_names,
        cmap="Blues",
        axes=ax[0][idx],
        show=False,
        extrapolate="local"
        #,vlim=(0,1)
    #     sphere=0.09
    )
    ax[0][idx].set_title(f"MinMax: {i}")
    
for idx, i in enumerate(methods):
    scaler= RobustScaler()
    scaled_avg_grad= scaler.fit_transform(avg_grad[i].sum(1).reshape(-1,1))
#     scaled_avg_grad= avg_grad[i].sum(1)

    robust_im, _= mne.viz.plot_topomap(
        scaled_avg_grad.reshape(-1),
        info,
        ch_type= "eeg",
        sensors=True,
        names=channel_names,
        cmap="Blues",
        axes=ax[1][idx],
        show=False,
        extrapolate="local",
#         vlim=(0,None)
    #     sphere=0.09
    )
    ax[1][idx].set_title(f"Robust: {i}")
    
fig.set_figheight(25)
fig.set_figwidth(25)
# fig.add_axes(ax)
cbar_ax= fig.add_axes([.92,0.66, 0.02,0.18])
clb= fig.colorbar(minmax_im, cax=cbar_ax)
clb.set_label("Importance Estimate", rotation=270,labelpad=20, fontsize=18)

cbar_ax= fig.add_axes([.92,0.22, 0.02,0.18])
clb= fig.colorbar(robust_im, cax=cbar_ax)
clb.set_label("Importance Estimate", rotation=270,labelpad=20, fontsize=18)

for tt in plt.findobj(fig, matplotlib.text.Text):
    if tt.get_text() in channel_names:
        tt.set_fontsize(14)
# plt.tight_layout()